In [29]:
import cv2
import numpy as np

def detect_dolphins(video_path, threshold= 5.3e7 , fps =30):
    # Ouvrir la vidéo
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur lors de l'ouverture de la vidéo.")
        return

    frames = []
    instants_dauphins = []

    frame_number = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir en niveau de gris pour simplifier l'analyse
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Si nous avons plus de 10 images, supprimer la plus ancienne
        if len(frames) > 10:
            frames.pop(0)

        # Comparer l'image actuelle avec les dix précédentes
        for prev_frame in frames:
            diff = cv2.absdiff(gray, prev_frame)
            diff_sum = np.sum(diff)
            
            if diff_sum > threshold:
                instants_dauphins.append(frame_number//fps)
                break

        frames.append(gray)
        frame_number += 1

    cap.release()
    return instants_dauphins

video_path = "/home/alexis/Desktop/short1.mp4"
instants = detect_dolphins(video_path)
print("Dauphins détectés aux instants:", instants)

Dauphins détectés aux instants: [68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 93, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 95, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96, 96]


In [50]:
import cv2
import pandas as pd

def find_dolphins_in(video_path, duration_threshold = 1.71, contour_threshold=1850):
    """
    Looks for big moving objects.

    :param video_path: Path to the video file
    :param duration_threshold: Duration in seconds to qualify as motion
    :param contour_threshold : threshold for the size of the object we want to detect
    :return: The motion event times
    """
  
    cap = cv2.VideoCapture(video_path)
    # Get the frames per second (fps) of the video
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Initialize background frame for background subtraction
    ret, frame = cap.read()
    if not ret:
        print("Failed to read the video")
        cap.release()
        exit()

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (21, 21), 0)

    # DataFrame to store motion data
    motion_list = [None, None]
    motion_times = []
    rows_list = []  # list to collect dictionaries for creating DataFrame

    frame_count = 0

    while True:
        # Read frame from the camera
        ret, frame = cap.read()
        if not ret:
            break  # Exit loop if no more frames are available

        motion = 0

        # Preprocess the frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        # Compute the absolute difference between the current frame and background frame
        diff_frame = cv2.absdiff(gray_frame, gray)

        # Apply thresholding to get the foreground mask
        thresh_frame = cv2.threshold(diff_frame, 30, 255, cv2.THRESH_BINARY)[1]
        thresh_frame = cv2.dilate(thresh_frame, None, iterations=2)

        # Find contours of moving objects
        cnts, _ = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in cnts:
            if cv2.contourArea(contour) < contour_threshold:
                continue
            motion = 1

        # Append status of motion
        motion_list.append(motion)
        motion_list = motion_list[-2:]

        # Append Start time of motion
        if motion_list[-1] == 1 and motion_list[-2] == 0:
            motion_times.append(frame_count / fps)

        # Append End time of motion
        if motion_list[-1] == 0 and motion_list[-2] == 1:
            motion_times.append(frame_count / fps)

        frame_count += 1

    for i in range(0, len(motion_times)-1, 2):
        duration = motion_times[i + 1] - motion_times[i]
        if duration > duration_threshold:
            rows_list.append({"Start": motion_times[i], "End": motion_times[i + 1]})

    if rows_list:
        df = pd.concat([pd.DataFrame([row]) for row in rows_list], ignore_index=True)
    else:
        print("No data to concatenate. Returning an empty DataFrame.")
        df = pd.DataFrame()

    # Release video capture object
    cap.release()

    return df

# Example usage
video_path = "/home/alexis/Desktop/short.mp4"
motion_df = find_dolphins_in(video_path)
print(motion_df.head())




# a faire : tester en divisant le contour threshold npar 16 et regarder les correspondances de timing pour short.
# mettre lextrait video dans un fichier separ


        Start         End
0   91.224467   97.430667
1  102.502400  105.905800
2  398.131067  400.066333


In [31]:
import os
import pandas as pd

def process_all_videos(directory_path, processing_function, checkpoint_file="checkpoint.csv"):
    # Check if the directory exists
    if not os.path.exists(directory_path):
        print("Directory does not exist:", directory_path)
        return

    # Try to load progress from the checkpoint file
    processed_videos = []
    if os.path.exists(checkpoint_file):
        processed_videos = pd.read_csv(checkpoint_file)['Filename'].tolist()
        print("Resuming from checkpoint, skipping files:", processed_videos)

    all_dfs = []

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        if os.path.isfile(file_path) and file_path.endswith(".mp4"):
            if filename in processed_videos:
                print(f"Skipping already processed file: {filename}")
                continue

            print(f"Processing file: {filename}")
            df = processing_function(file_path)
            all_dfs.append((filename, df))

            # Update checkpoint file
            processed_videos.append(filename)
            pd.DataFrame({'Filename': processed_videos}).to_csv(checkpoint_file, index=False)

    return all_dfs

# Example usage
directory_path = "/home/alexis/Desktop/"
processed_videos = process_all_videos(directory_path, find_dolphins_in)

# Print the results for each video
for video_name, df in processed_videos:
    print(f"Results for {video_name}:")
    if df is not None and not df.empty:
        print(df.head())
    else:
        print("No data returned or DataFrame is empty")

Resuming from checkpoint, skipping files: ['short.mp4']
Skipping already processed file: short.mp4
Processing file: Exp_12_Oct_2023_1245_cam_all.mp4


ValueError: No objects to concatenate

In [27]:
import os
import pandas as pd

def process_all_videos(directory_path, processing_function):
    # Check if the directory exists
    if not os.path.exists(directory_path):
        print("Directory does not exist:", directory_path)
        return

    # List to hold all DataFrames
    all_dfs = []

    # Loop through each file in the directory
    for filename in os.listdir(directory_path):
        # Construct full file path
        file_path = os.path.join(directory_path, filename)
        
        # Check if it's a file and not a directory
        if os.path.isfile(file_path):
            # Assuming you want to process only .mp4 files
            if file_path.endswith(".mp4"):
                # Apply the processing function
                df = processing_function(file_path)

                # Add the DataFrame to the list, along with filename for reference
                all_dfs.append((filename, df))

    return all_dfs

# Example usage
directory_path = "/home/alexis/Desktop/"
processed_videos = process_all_videos(directory_path, find_dolphins_in)

# Print the results for each video
for video_name, df in processed_videos:
    print(f"Results for {video_name}:")
    print(df.head())


KeyboardInterrupt: 

In [41]:
import os
import pandas as pd

class VideoProcessor:
    def __init__(self, directory_path, processing_function, results_file="processed_results.csv"):
        self.directory_path = directory_path
        self.processing_function = processing_function
        self.results_file = results_file
        self.all_results = self._load_results()

    def _load_results(self):
        if os.path.exists(self.results_file):
            return pd.read_csv(self.results_file)
        else:
            return pd.DataFrame(columns=['Filename', 'Data'])

    def has_been_processed(self, filename):
        return filename in self.all_results['Filename'].values

    def process_videos(self):
        for filename in os.listdir(self.directory_path):
            file_path = os.path.join(self.directory_path, filename)
            if os.path.isfile(file_path) and file_path.endswith(".mp4") and not self.has_been_processed(filename):
                print(f"Processing file: {filename}")
                df = self.processing_function(file_path)
                self._update_results(filename, df)

        # Final update to results file
        self._write_results()

    def _update_results(self, filename, df):
        if df is not None and not df.empty:
            df['Filename'] = filename
            self.all_results = pd.concat([self.all_results, df], axis=0)

    def _write_results(self):
        self.all_results.to_csv(self.results_file, index=False)

processor = VideoProcessor("/home/alexis/Desktop/video", find_dolphins_in)
processor.process_videos()


Processing file: short1.mp4
Processing file: testvidéo.mp4


ValueError: No objects to concatenate

In [37]:
import os
import pandas as pd

def process_all_videos(directory_path, processing_function, results_file="processed_results.csv"):
    if not os.path.exists(results_file):
        all_results = pd.DataFrame(columns=['Filename', 'Data'])
    else:
        all_results = pd.read_csv(results_file)

    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)

        if os.path.isfile(file_path) and file_path.endswith(".mp4") and filename not in all_results['Filename'].tolist():
            print(f"Processing file: {filename}")
            df = processing_function(file_path)

            if df is not None and not df.empty:
                df['Filename'] = filename
                all_results = all_results.append(df, ignore_index=True)

                # Update the results file periodically
                all_results.to_csv(results_file, index=False)

    print("Processing complete.")

# Usage
directory_path = "/home/alexis/Desktop/"
process_all_videos(directory_path, find_dolphins_in)


Processing file: short.mp4


AttributeError: 'DataFrame' object has no attribute 'append'